In [ ]:
torchserve --stop
sleep 1s
torchserve --start --model-store model-archive/model_store/ --workflow-store model-archive/wf_store/ --ncs
sleep 1s
curl -X POST "localhost:7081/workflows?url=ocr.war"
curl -X POST   -H "Content-Type: application/json; charset=utf-8"   -d @sample_b64.json   localhost:7080/wfpredict/ocr -o output1.json &\
curl -X POST   -H "Content-Type: application/json; charset=utf-8"   -d @sample_b64.json   localhost:7080/wfpredict/ocr -o output2.json &\
curl -X POST   -H "Content-Type: application/json; charset=utf-8"   -d @sample_b64.json   localhost:7080/wfpredict/ocr -o output3.json &\
curl -X POST   -H "Content-Type: application/json; charset=utf-8"   -d @sample_b64.json   localhost:7080/wfpredict/ocr -o output4.json &\

In [23]:
PROJECT_ID = "pytorch-d2p"
BUCKET_NAME = "gs://d2p-ocr"
REGION = "us-central1"

In [3]:
hostname = "gcr.io"
tag = "latest"

image_name_serve = "ocr-gpu"
custom_container_image_uri_serve = f"{hostname}/{PROJECT_ID}/{image_name_serve}:{tag}"

In [4]:
! echo $custom_container_image_uri_serve
! docker build -t $custom_container_image_uri_serve -f Dockerfile .

gcr.io/pytorch-d2p/ocr-gpu:latest

Step 1/16 : FROM pytorch/torchserve:latest-gpu
 ---> bfa7ebfb07b8
Step 2/16 : USER root
 ---> Using cache
 ---> db541cd3d75d
Step 3/16 : RUN apt-get update; apt-get install ffmpeg curl vim libsm6 libxext6 -y
 ---> Using cache
 ---> 2fe4ddbfa59c
Step 4/16 : RUN printf "\ninference_address=http://0.0.0.0:7080" >> /home/model-server/config.properties
 ---> Using cache
 ---> 43e5262238a6
Step 5/16 : RUN printf "\nmanagement_address=http://0.0.0.0:7081" >> /home/model-server/config.properties
 ---> Using cache
 ---> 34c25421eb45
Step 6/16 : RUN printf "\nmetrics_address=http://0.0.0.0:7082" >> /home/model-server/config.properties
 ---> Using cache
 ---> 5c8d692defda
Step 7/16 : COPY ./dockerd-ep.sh /home/model-server/dockerd-ep.sh
 ---> Using cache
 ---> 0223bc2a32b8
Step 8/16 : RUN chmod +x /home/model-server/dockerd-ep.sh
 ---> Using cache
 ---> 046114ebf4b7
Step 9/16 : USER model-server
 ---> Using cache
 ---> 5402865ac59f
Step 10/16 : RUN pip3 install 

In [11]:
! docker run --gpus all -d --name test-ocr8 -p 7080:7080 -p 7081:7081 -p 7082:7082 $custom_container_image_uri_serve
! sleep 15s

049934b3ef826c3d8c22f7dcd558317fe02fddebf21491e99e905884128bbcb5


In [13]:
! curl http://localhost:7080/ping
! curl -X POST   -H "Content-Type: application/json; charset=utf-8"   -d @sample_b64.json   localhost:7080/wfpredict/ocr

{
  "status": "Healthy"
}
{
  "predictions": [
    {
      "bbox": [
        [
          0.392,
          0.06111111111111111
        ],
        [
          0.612,
          0.07222222222222222
        ],
        [
          0.606,
          0.13518518518518519
        ],
        [
          0.386,
          0.12407407407407407
        ]
      ],
      "ocr": [
        "Receipt",
        0.9972578627722604
      ],
      "key": [
        "#other",
        1
      ]
    },
    {
      "bbox": [
        [
          0.283,
          0.14814814814814814
        ],
        [
          0.38,
          0.14814814814814814
        ],
        [
          0.38,
          0.17037037037037037
        ],
        [
          0.283,
          0.17037037037037037
        ]
      ],
      "ocr": [
        "Adress",
        0.9997507532437643
      ],
      "key": [
        "#other",
        1
      ]
    },
    {
      "bbox": [
        [
          0.394,
          0.14814814814814814
        ],
      

In [19]:
! docker stop test-ocr8
! docker push $custom_container_image_uri_serve

test-ocr8
The push refers to repository [gcr.io/pytorch-d2p/ocr-gpu]





















latest: digest: sha256:bc8e98b47fb930fd258b168a08726a9ee038bdd02fb4d39f42d1d10e22c589c6 size: 5143


In [20]:
! gcloud container images list --repository $hostname/$PROJECT_ID

NAME
gcr.io/pytorch-d2p/ocr-gpu


In [45]:
# ! pip install google-cloud-aiplatform
from google.cloud import aiplatform
REGION='asia-southeast1'
aiplatform.init(
    project=PROJECT_ID,
    staging_bucket=BUCKET_NAME,
    location=REGION,
)

In [46]:
display_name = image_name_serve + '-v1'
description = 'PyTorch based optical character recognizer served using a custom torchserve workflow on GPU'
model = aiplatform.Model.upload(
    display_name=display_name,
    description=description,
    serving_container_image_uri=custom_container_image_uri_serve,
    serving_container_ports=[7080],
    serving_container_predict_route=f"/wfpredict/ocr",
    serving_container_health_route="/ping",
)
model.wait()

print(model.display_name)
print(model.resource_name)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/576632640300/locations/asia-southeast1/models/2491651678214815744/operations/4767768291059957760
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/576632640300/locations/asia-southeast1/models/2491651678214815744
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/576632640300/locations/asia-southeast1/models/2491651678214815744')
ocr-gpu-v1
projects/576632640300/locations/asia-southeast1/models/2491651678214815744


In [47]:
endpoint_display_name = f"{image_name_serve}-deploy"
endpoint = aiplatform.Endpoint.create(display_name=endpoint_display_name)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/576632640300/locations/asia-southeast1/endpoints/283797145268518912/operations/5004207271496908800
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/576632640300/locations/asia-southeast1/endpoints/283797145268518912
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/576632640300/locations/asia-southeast1/endpoints/283797145268518912')


In [50]:
traffic_percentage = 100
machine_type = "n1-standard-4"
accelerator_type = "NVIDIA_TESLA_T4"
min_replica_count = 1
max_replica_count = 3
sync = True

model.deploy(
    endpoint=endpoint,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=1,
    traffic_percentage=traffic_percentage,
    # sync=sync,
)

model.wait()

INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/576632640300/locations/asia-southeast1/endpoints/283797145268518912
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/576632640300/locations/asia-southeast1/endpoints/283797145268518912/operations/4524573911181950976


ResourceExhausted: 429 The following quotas are exceeded: CustomModelServingT4GPUsPerProjectPerRegion

In [36]:
import base64

def convert_b64(input_file_name):
    """Open image and convert it to Base64"""
    with open(input_file_name, "rb") as input_file:
        jpeg_bytes = base64.b64encode(input_file.read()).decode("utf-8")
    return jpeg_bytes

In [38]:
image_file_name = "../test_images/receipt.jpg"
instance = {"data": {"b64": convert_b64(image_file_name)}}
prediction = endpoint.predict(instances=[instance])
print(f"Prediction response: \n\t{prediction}")

ServiceUnavailable: 503 502:Bad Gateway

In [32]:
from typing import Dict

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instance_dict: Dict,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instance = json_format.ParseDict(instance_dict, Value())
    instances = [instance]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))


In [34]:
image_file_name = "../test_images/receipt.jpg"
instance = {"data": {"b64": convert_b64(image_file_name)}}
predict_custom_trained_model_sample(
    project="284519662159",
    endpoint_id="7765058978609889280",
    location="us-central1",
    instance_dict=instance
)

response
 deployed_model_id: 735819569586765824
 prediction: {'key': ['#other', 1.0], 'ocr': ['VectorStock', '0.9765939'], 'bbox': [[0.03, 0.95], [0.217, 0.9481481481481482], [0.218, 0.9768518518518519], [0.03, 0.9796296296296296]]}
 prediction: {'bbox': [[0.647, 0.7111111111111111], [0.718, 0.7111111111111111], [0.718, 0.7342592592592593], [0.647, 0.7342592592592593]], 'key': ['#other', 1.0], 'ocr': ['84.80', '0.9978407']}
 prediction: {'key': ['#other', 1.0], 'bbox': [[0.272, 0.7092592592592593], [0.383, 0.712037037037037], [0.382, 0.7333333333333333], [0.271, 0.7314814814814815]], 'ocr': ['Balance', '0.9269944']}
 prediction: {'bbox': [[0.658, 0.6777777777777778], [0.719, 0.6777777777777778], [0.719, 0.700925925925926], [0.658, 0.700925925925926]], 'ocr': ['8.00', '0.9957166'], 'key': ['#other', 1.0]}
 prediction: {'bbox': [[0.273, 0.6768518518518518], [0.404, 0.6796296296296296], [0.404, 0.700925925925926], [0.272, 0.6990740740740741]], 'key': ['#other', 1.0], 'ocr': ['Sales Tax', 